#**Photo Response Non-Uniformity (PRNU)** - An Overview#

**What is it?:**
* a scant residual introduced by the sensor of the camera of your devices
* unique
* sensible to spatial transformations 


**Where is it used?**
* Device Identification
* Tampering Detection

**Main Problems:**
* Spatial Transformations (i.e. crop, up-scaling, down-scaling, radial corrections, and many more desynchronize the PRNU and make it unreliable
* JPEG compressions reduce its reliablity

##How do you extract the PRNU?##

- Wavelet Denoiser NoiseExtractFromImage()
- Zero Mean by Row and Columns
- Wiener Filter in DFT


In [ ]:
#install requirements
!pip install scipy == 1.4.1
!pip install matplotlib == 3.3.3
!pip install opencv-python == 4.1.2.30

[info and papers about PRNU](http://dde.binghamton.edu/download/camera_fingerprint/)

In [ ]:
import os
!pip install binghamton-camerafp
# The package has been removed by author's request.
# Visit the aforementioned website for a copy of the library.

In [ ]:
os.chdir('code/CameraFingerprint')

##**Example1 Device Identification**##
**[Match between Camera Fingerprint and PRNU of the Image]**

In [ ]:
import camerafp.functions as Fu
import camerafp.filter as Ft
import camerafp.get_fingerprint as gF
import camerafp.maindir as md
import camerafp.extra_utils as eu
import numpy as np
import os
import cv2 as cv

# composing the Fingerprint from the images of Camera A
im1 = 'Images' + os.sep + 'P1.jpg'
im2 = 'Images' + os.sep + 'P2.jpg'
im3 = 'Images' + os.sep + 'P3.jpg'
Images = [im1, im2, im3]

RP, _, _ = gF.getFingerprint(Images)
RP = Fu.rgb2gray1(RP)
sigmaRP = np.std(RP)
Fingerprint = Fu.WienerInDFT(RP, sigmaRP)

#extracting the PRNU from another image of Camera A
imx = 'Images' + os.sep + 'Pxxx.jpg'
Noisex = Ft.NoiseExtractFromImage(imx, sigma=2.)
Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))

# The optimal detector (see publication "Large Scale Test of Sensor Fingerprint Camera Identification")
Ix = cv.cvtColor(cv.imread(imx),  # image in BGR format
                 cv.COLOR_BGR2GRAY)

#compute the peak of correlation energy PCE (the measure of similarity between Noisex and Fingerprint)
C = Fu.crosscorr(Noisex, np.multiply(Ix, Fingerprint))
det, det0 = md.PCE(C)
for key in det.keys(): print("{0}: {1}".format(key, det[key]))
eu.mesh(C)

##**Example2 Device Identification**##
 **[mis-Match between Camera Fingerprint and PRNU of the Image]**

In [ ]:
if not os.path.exists('im_H0.jpg'):
    !wget -O im_H0.jpg "https://drive.google.com/uc?export=download&id=1tpiLPOVvKiBKYo6wfkoXCa89Vye9u6dO"

In [ ]:

# composing the Fingerprint from the images of Camera A
im1 = 'Images' + os.sep + 'P1.jpg'
im2 = 'Images' + os.sep + 'P2.jpg'
im3 = 'Images' + os.sep + 'P3.jpg'
Images = [im1, im2, im3]

RP, _, _ = gF.getFingerprint(Images)
RP = Fu.rgb2gray1(RP)
sigmaRP = np.std(RP)
Fingerprint = Fu.WienerInDFT(RP, sigmaRP)

#extract the PRNU from an image of Camera B (i.e. Camera A and Camera B are NOT the same device)
imx = 'im_H0.jpg'
Noisex = Ft.NoiseExtractFromImage(imx, sigma=2.)
Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))

# The optimal detector (see publication "Large Scale Test of Sensor Fingerprint Camera Identification")
Ix = cv.cvtColor(cv.imread(imx),  # image in BGR format
                 cv.COLOR_BGR2GRAY)

#compute the peak of correlation energy PCE (the measure of similarity between Noisex and Fingerprint)
C = Fu.crosscorr(Noisex, np.multiply(Ix, Fingerprint))
det, det0 = md.PCE(C)
for key in det.keys(): print("{0}: {1}".format(key, det[key]))
eu.mesh(C)

##**Example3: Tampering Detection**##

In [ ]:
!pip install gdown

In [ ]:
import os

#import test image
if not os.path.exists('IMG_0047.jpg'):
    !wget -O IMG_0047.jpg "https://drive.google.com/uc?export=download&id=1HTD86ybdjfBvg7AOyxaQfxyB1YphU5Uk"
#import camera fingerprint
if not os.path.exists('Fingerprint_CanonEOS1200d.dat'):
    !gdown https: // drive.google.com / uc?id=1qy2R4AsxkOnkOULBkMFR9SzT7QifTr5C

In [ ]:
import matplotlib.image as mpimg

#read camera fingerprint
Fingerprint = np.genfromtxt('Fingerprint_CanonEOS1200d.dat')
print('SIZE CAMERA FINGERPRINT: ', np.shape(Fingerprint))

imx = 'IMG_0047.jpg'
img = mpimg.imread(imx)
#extract PRNU from  an image presumely taken with the same camera of Fingerprint
Noisex = Ft.NoiseExtractFromImage(imx, sigma=2.)
Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))
print('SIZE PRNU IMAGE: ', np.shape(Noisex))

if np.shape(Noisex) == np.shape(Fingerprint):
    print('Camera Fingerprint and Image PRNU size are the same. CASE: Perfectly aligned!')
    shift_range = [0, 0]
else:
    print('Camera Fingerprint and Image PRNU size are NOT the same. CASE: NOT perfectly aligned!')
    Noisex1 = np.zeros_like(Fingerprint)
    Noisex1[:Noisex.shape[0], :Noisex.shape[1]] = Noisex
    Noisex = Noisex1

#divide image and fingerprint by blocks and compute the PCE of each block
blocks_x = np.arange(0, Noisex.shape[0], 64)
blocks_y = np.arange(0, Noisex.shape[1], 64)
PCE_map = np.zeros((len(blocks_x), len(blocks_y)))
for y in range(0, len(blocks_y)):
    for x in range(0, len(blocks_x)):
        block_Noisex = Noisex[blocks_x[x]:blocks_x[x] + 64, blocks_y[y]:blocks_y[y] + 64]
        block_Fingerprint = Fingerprint[blocks_x[x]:blocks_x[x] + 64, blocks_y[y]:blocks_y[y] + 64]
        C = Fu.crosscorr(block_Noisex, block_Fingerprint)
        det, det0 = md.PCE(C)
        PCE_map[x, y] = det['PCE']
import matplotlib.pyplot as plt

plt.imshow(PCE_map)
plt.title('Detection PCE-map')
plt.show()
plt.imshow(img)
plt.title('Tampered Image')
plt.show()

##**The spatial transformation's problem**##
Example: Radial Correction

In [ ]:
import os

#import test image
if not os.path.exists('im5.jpg'):
    !wget -O im5.jpg "https://drive.google.com/uc?export=download&id=1Q9lDMcG0-sps-GLw2NuSgaa_CeLuKGv2"
#import camera fingerprint
if not os.path.exists('FINGERPRINT_CanonSX230HS_focal70.dat'):
    !wget -O FINGERPRINT_CanonSX230HS_focal70.dat "https://drive.google.com/uc?export=download&id=1xJnSw-lSouswj5EpDzkWR2oGJDfOuhEe"


In [ ]:
#read camera fingerprint
Fingerprint = np.genfromtxt('FINGERPRINT_CanonSX230HS_focal70.dat')
print('SIZE CAMERA FINGERPRINT: ', np.shape(Fingerprint))

#extract PRNU from image taken with the same camera of Fingerprint
imx = 'im5.jpg'
Noisex = Ft.NoiseExtractFromImage(imx, sigma=2.)
Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))
print('SIZE PRNU IMAGE: ', np.shape(Noisex))

if np.shape(Noisex) == np.shape(Fingerprint):
    print('Camera Fingerprint and Image PRNU size are the same. CASE: Perfectly aligned!')
    shift_range = [0, 0]
    C = Fu.crosscorr(Noisex, Fingerprint)
else:
    print('Camera Fingerprint and Image PRNU size are NOT the same. CASE: NOT perfectly aligned!')
    Noisex1 = np.zeros_like(Fingerprint)
    Noisex1[:Noisex.shape[0], :Noisex.shape[1]] = Noisex
    shift_range = [Fingerprint.shape[0] - Noisex.shape[0], Fingerprint.shape[1] - Noisex.shape[1]]
    C = Fu.crosscorr(Noisex1, Fingerprint)

#compute the peak of correlation energy PCE (the measure of similarity between Noisex and Fingerprint)
det, det0 = md.PCE(C, shift_range=shift_range)
print("{0}: {1}".format('PCE', det['PCE']))
print("{0}: {1}".format('PeakLocation', det['PeakLocation']))
eu.mesh(C)

Example: Down-scaling

In [ ]:
import os

#import test image
if not os.path.exists('im_downscale.jpg'):
    !wget -O im_downscale.jpg "https://drive.google.com/uc?export=download&id=1kmPotOdLBSZGKPPFHmVjaXuyRWxQVvLS"
#import camera fingerprint
if not os.path.exists('FINGERPRINT_D01.dat'):
    !wget -O FINGERPRINT_D01.dat "https://drive.google.com/uc?export=download&id=1DmNH1hLzsQ_rFcF2DB7VvNHiaZhoy7HH"

In [ ]:
#read camera fingerprint
Fingerprint = np.genfromtxt('FINGERPRINT_D01.dat')
print('SIZE CAMERA FINGERPRINT: ', np.shape(Fingerprint))

#extract PRNU from image taken with the same camera of Fingerprint
imx = 'im_downscale.jpg'
Noisex = Ft.NoiseExtractFromImage(imx, sigma=2.)
Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))
print('SIZE PRNU IMAGE: ', np.shape(Noisex))

if np.shape(Noisex) == np.shape(Fingerprint):
    print('Camera Fingerprint and Image PRNU size are the same. CASE: Perfectly aligned!')
    shift_range = [0, 0]
    C = Fu.crosscorr(Noisex1, Fingerprint)
else:
    print('Camera Fingerprint and Image PRNU size are NOT the same. CASE: NOT perfectly aligned!')
    Noisex1 = np.zeros_like(Fingerprint)
    Noisex1[:Noisex.shape[0], :Noisex.shape[1]] = Noisex
    shift_range = [Fingerprint.shape[0] - Noisex.shape[0], Fingerprint.shape[1] - Noisex.shape[1]]
    C = Fu.crosscorr(Noisex1, Fingerprint)

det, det0 = md.PCE(C, shift_range=shift_range)
#for key in det.keys(): print("{0}: {1}".format(key, det[key]))
print("{0}: {1}".format('PCE', det['PCE']))
print("{0}: {1}".format('PeakLocation', det['PeakLocation']))
eu.mesh(C)


**Exercise 1**

Determine which image was taken with the same device of the images used to compose the camera fingerprint.

**TIP**: There's only one image and it has the highest PCE value.

In [ ]:
import os

#import test image
if not os.path.exists('im1.jpg'):
    !wget -O im1.jpg "https://drive.google.com/uc?export=download&id=1V3uJT-m2uPXlxlQuGRY6Z5CJzNl89Sgp"
#import test image
if not os.path.exists('im2.jpg'):
    !wget -O im2.jpg "https://drive.google.com/uc?export=download&id=1JYaAlb4lg2cYq8RSJhnyKg0YNixBhGLg"
#import test image
if not os.path.exists('im3.jpg'):
    !wget -O im3.jpg "https://drive.google.com/uc?export=download&id=1SMcJr3hwiBL2Z8oA_ougneBtlYE-4lv_"
#import test image
if not os.path.exists('im4.jpg'):
    !wget -O im4.jpg "https://drive.google.com/uc?export=download&id=1OcvvchD6iXKG5NulA8RxR61S8J9k9GwU"

In [ ]:
# extracting Fingerprint from same size images in a path
im1 = 'Images' + os.sep + 'P1.jpg'
im2 = 'Images' + os.sep + 'P2.jpg'
im3 = 'Images' + os.sep + 'P3.jpg'
Images = [im1, im2, im3]

RP, _, _ = gF.getFingerprint(Images)
RP = Fu.rgb2gray1(RP)
sigmaRP = np.std(RP)
Fingerprint = Fu.WienerInDFT(RP, sigmaRP)
#import images and extract their PRNU
imx = ['im1.jpg', 'im2.jpg', 'im3.jpg', 'im4.jpg']
'''
YOUR CODE
'''
#COMPUTE AND PRINT ONE BY ONE THE PCE VALUES TO DETECT THE IMAGE TAKEN WITH THE
#SAME CAMERA OF FINGERPRINT
'''
YOUR CODE
'''

##**Exercise 2**#
Find the parameter able to reverse the **down-scaling** spatial transformation and maximize the final PCE value (~70 is the goal). \\


In [ ]:
import os

#import test image
if not os.path.exists('frame.png'):
    !wget -O frame.png "https://drive.google.com/uc?export=download&id=1ZKxSK3VKMJCKtgEmSjG4XXBeaFNRqCLe"


In [ ]:
from skimage.transform import rescale

#read camera fingerprint
Fingerprint = np.genfromtxt('FINGERPRINT_D01.dat')
print('SIZE CAMERA FINGERPRINT: ', np.shape(Fingerprint))

#extract PRNU from image
imx = 'frame.png'
Noisex = Ft.NoiseExtractFromImage(imx, sigma=2.)
Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))
print('SIZE PRNU IMAGE: ', np.shape(Noisex))

if np.shape(Noisex) == np.shape(Fingerprint):
    print('Camera Fingerprint and Image PRNU size are the same. CASE: Perfectly aligned!')
    shift_range = [0, 0]
    C = Fu.crosscorr(Noisex1, Fingerprint)
else:
    print('Camera Fingerprint and Image PRNU size are NOT the same. CASE: NOT perfectly aligned!')
    Noisex1 = np.zeros_like(Fingerprint)
    Noisex1[:Noisex.shape[0], :Noisex.shape[1]] = Noisex
    shift_range = [Fingerprint.shape[0] - Noisex.shape[0], Fingerprint.shape[1] - Noisex.shape[1]]
    C = Fu.crosscorr(Noisex1, Fingerprint)

det, det0 = md.PCE(C, shift_range=shift_range)
print('PCE value before down-scaling inversion (i.e. upscaling)')
print("{0}: {1}".format('PCE', det['PCE']))

k = 1
while k >= 0.49999:
    #rescale your image
    Noisex_up = rescale(Noisex, 1 / k)
    '''
    YOUR CODE
    '''

**Possibily there's a smarter way to find k (in one shot)! Can you guess it?** \\
TIP: the camera fingerprint is composed with images, instead the PRNU is extracted from a video frame taken with the same device of the camera fingerprint. \\
If you find the solution, **let us know** :-)

##**Exercise 3**#
Find the parameters able to reverse the **radial correction** spatial transformation applied to the test image. GOAL: PCE ~250 \\


Useful functions to reverse the radial correction : -------- )

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
import warnings

warnings.filterwarnings('ignore')


def radial_cordinates(M, N):
    center = [M / 2, N / 2]
    xi, yi = np.meshgrid(np.arange(M), np.arange(N))
    xt = xi - center[0]
    yt = yi - center[1]
    r = np.sqrt(xt ** 2 + yt ** 2)
    theta = np.arctan2(xt, yt)
    R = np.sqrt(center[0] ** 2 + center[1] ** 2)
    r = r / R
    return r, theta, R, xi, yi, center, xt, yt


def distortfct(r, k):
    s = r * (1 - k * (r ** 2) + 3 * (k ** 2) * (r ** 4))
    return s


def imdistcorrect(img, k, r, theta, R, xi, yi, M, N):
    s = distortfct(r, k)
    s2 = s * R
    v = s2 * np.cos(theta)
    u = s2 * np.sin(theta)
    #PIPELINE BARRELL
    if np.amin(np.round(v + np.abs(np.amax((N)) // 2))) < 0:
        print('barrell')
        v = np.round(v + np.abs(np.amax((N)) // 2))
        u = np.round(u + np.abs(np.amax((M)) // 2))
        u = u.astype(np.int32)
        v = v.astype(np.int32)
        dist = np.zeros([np.max(v + 1), np.max(u + 1)])
        dist[yi[np.logical_and(v < N - 1, v > 0) * np.logical_and(u < M - 1, u > 0)], xi[
            np.logical_and(v < N - 1, v > 0) * np.logical_and(u < M - 1, u > 0)]] = img[
            v[np.logical_and(v < N - 1, v > 0) * np.logical_and(u < M - 1, u > 0)], u[
                np.logical_and(v < N - 1, v > 0) * np.logical_and(u < M - 1, u > 0)]]  #yi, xi]#[v, u]  # [yi, xi]
    else:
        #PIPELINE PINCUSHION
        print('pincushion')
        v = np.round(v + np.abs(np.amax((N)) // 2))
        u = np.round(u + np.abs(np.amax((M)) // 2))
        u = u.astype(int)
        v = v.astype(int)
        dist = np.zeros([np.max(yi + 1), np.max(xi + 1)])
        dist[yi, xi] = img[v, u]

    dist = bilinear_interpolation(dist)
    size_dist = np.shape(dist)
    if size_dist[0] * size_dist[1] > M * N:
        dist = crop_center(dist, M, N)

    return dist


def bilinear_interpolation(img):
    '''
    This function fill the 0 values by interpolating them
    '''
    x = np.arange(len(img))
    img_new = np.zeros(img.shape)
    aux_x = []
    aux_y = []
    for i in range(len(img[0]) - 1):
        ix = np.where(img[:, i] != 0)
        if (len(ix[0]) != 0):
            f = interp1d(x[ix], img[ix, i], fill_value='extrapolate')
            img_new[x[ix[0][0]:ix[0][-1]], i] = f(x[ix[0][0]:ix[0][-1]])
            aux_y.append(x[0:ix[0][0]])
            aux_y.append(x[ix[0][-1]:len(img[0])])
            aux_x.append(i * np.ones(len(x[0:ix[0][0]]) + len(x[ix[0][-1]:len(img[0])])))
    x = np.arange(len(img[0]))
    for i in range(0, (len(img) - 1)):
        ix = np.where(img[i, :] != 0)
        if (len(ix[0]) != 0):
            f = interp1d(x[ix], img[i, ix], fill_value='extrapolate')
            img_new[i, x[ix[0][0]:ix[0][-1]]] = f(x[ix[0][0]:ix[0][-1]])
            aux_x.append(x[0:ix[0][0]])
            aux_x.append(x[ix[0][-1]:len(img[0])])
            aux_y.append(i * np.ones(len(x[0:ix[0][0]]) + len(x[ix[0][-1]:len(img[0])])))
    aux_x = np.concatenate(np.array(aux_x)).astype(int)
    aux_y = np.concatenate(np.array(aux_y)).astype(int)
    img_new[aux_y, aux_x] = 0
    return img_new


def crop_center(img, cropx, cropy):
    startx = 0
    starty = 0
    return img[starty:starty + cropy, startx:startx + cropx]

**YOUR CODE**

In [ ]:
#read camera fingerprint
Fingerprint = np.genfromtxt('FINGERPRINT_CanonSX230HS_focal70.dat')
print('SIZE CAMERA FINGERPRINT: ', np.shape(Fingerprint))

#extract PRNU from image
imx = 'im5.jpg'
Noisex = Ft.NoiseExtractFromImage(imx, sigma=2.)
Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))
print('SIZE PRNU IMAGE: ', np.shape(Noisex))

if np.shape(Noisex) == np.shape(Fingerprint):
    print('Camera Fingerprint and Image PRNU size are the same. CASE: Perfectly aligned!')
    shift_range = [0, 0]
    C = Fu.crosscorr(Noisex, Fingerprint)
else:
    print('Camera Fingerprint and Image PRNU size are NOT the same. CASE: NOT perfectly aligned!')
    Noisex1 = np.zeros_like(Fingerprint)
    Noisex1[:Noisex.shape[0], :Noisex.shape[1]] = Noisex
    shift_range = [Fingerprint.shape[0] - Noisex.shape[0], Fingerprint.shape[1] - Noisex.shape[1]]
    C = Fu.crosscorr(Noisex1, Fingerprint)
det, det0 = md.PCE(C, shift_range=shift_range)
print('PCE value before Radial Correction Inversion')
print("{0}: {1}".format('PCE', det['PCE']))
print("{0}: {1}".format('PeakLocation', det['PeakLocation']))

#compute image size
M, N = [Noisex.shape[1], Noisex.shape[0]]
M = np.asarray(M)
N = np.asarray(N)
#compute polar coordinate (r, theta), half image diagonal size (R), 
#cartesian coordinates (xi, yi) and center coordinates
r, theta, R, xi, yi, center, _, _ = radial_cordinates(M, N)
#apply radial correction inversion with the following parameters
k = -0.22
while k < 0.23:
    Noisex_post = imdistcorrect(Noisex, k, r, theta, R, xi, yi, M, N)
    #find the correct k to invert the radial correctoin and maximize the PCE
    '''
    YOUR CODE
    '''

##**Exercise 4 | ONLY FOR BRAVE HEARTS**#
Find the parameters able to reverse a video stabilization (more info [here](https://drive.google.com/file/d/1sJiPOz0L1D9qU_pNL62ITZoZcaVqpSya/view?usp=sharing)) spatial transformation applied to the test frames. \\
**TIP**:  To understand if you did it the PCE values of the frames has to be larger than 40. If you succeed and you are able to do it without brute force search algorithm contact us! 

In [ ]:
import os

#import test image
if not os.path.exists('frame1.png'):
    !wget -O frame1.png "https://drive.google.com/uc?export=download&id=1_hIFS2LzuOMF7J-fDM_UksbIyOju2iSH"
if not os.path.exists('frame25.png'):
    !wget -O frame25.png "https://drive.google.com/uc?export=download&id=1lS-yQiDmH5x5fzWfVWJ58VskDTgHwRG5"
if not os.path.exists('frame50.png'):
    !wget -O frame50.png "https://drive.google.com/uc?export=download&id=1tCk8OYr-teiRY5axTrfYZjvzaIjlICLA"
if not os.path.exists('frame75.png'):
    !wget -O frame75.png "https://drive.google.com/uc?export=download&id=1QYpM2t50hJ-OfuhT1uYfjueZ3M_Rsp8G"
#import camera fingerprint
if not os.path.exists('FINGERPRINT_D02.dat'):
    !gdown https: // drive.google.com / uc?id=15c0kJCbET77a9Iin3MOURRl4PNi7A9U-


The [function](https://www.tensorflow.org/addons/api_docs/python/tfa/image/transform) you need to use! \\
[A short tutorial.](https://colab.research.google.com/github/tensorflow/addons/blob/master/docs/tutorials/image_ops.ipynb)

In [ ]:
!pip install tensorflow == 2.2.0
!pip install -U tensorflow-addons == 0.11.2

for more details on param refer to this slide

In [ ]:
param = [1.0, 0.0, -250, 0.0, 1.0, 0.0, 0.0, 0.0]  #just some random parameters -> [a0, a1, a2, b0, b1, b2, c0, c1]
transformed_Noisex = tfa.image.transform(Noisex, param, 'BILINEAR', [Fingerprint.shape[0], Fingerprint.shape[1]])

**some useful functions**

In [ ]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings

warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np


def circxcross2(mat1, mat2):
    a = mat1  # K
    b = mat2  # W
    sizem1 = (tf.shape(b)).numpy()  # sizeW
    sizem2 = (tf.shape(a)).numpy()  # sizeK

    if (sizem1[1] >= sizem2[1]) and (sizem1[0] >= sizem2[0]):
        a = mat2  # W
        b = mat1  # K
        sizem1 = (tf.shape(b)).numpy()
        sizem2 = (tf.shape(a)).numpy()

    a = a - tf.reduce_mean(a)
    a = a / tf.norm(a)
    b = b - tf.reduce_mean(b)
    b = b / tf.norm(b)
    paddings = tf.constant([[0, sizem2[0] - sizem1[0], ], [0, sizem2[1] - sizem1[1]], [0, 0]])
    b = tf.pad(b, paddings, "CONSTANT")
    tilted_b = tf.image.rot90(b, k=2)
    tilted_b = tf.cast(tf.squeeze(tilted_b), tf.complex64)
    a = tf.cast(tf.squeeze(a), tf.complex64)
    FF = tf.multiply(tf.signal.fft2d(a), tf.signal.fft2d(tilted_b))
    ret = tf.math.real(tf.signal.ifft2d(FF))
    return tf.roll(ret, shift=[1, 1], axis=[1, 1])


def pce2(cc: np.ndarray, ranges, neigh_radius: int = 2):
    """
    PCE VALUE
    """
    assert (cc.ndim == 2)
    assert (isinstance(neigh_radius, int))

    cc_inrange = cc[:ranges[0], :ranges[1]]
    max_idx = np.argmax(cc_inrange.flatten())
    max_y, max_x = np.unravel_index(max_idx, cc_inrange.shape)

    peak_height = cc[max_y, max_x]

    cc_nopeaks = cc.copy()
    cc_nopeaks[max_y - neigh_radius:max_y + neigh_radius, max_x - neigh_radius:max_x + neigh_radius] = 0

    pce_energy = np.mean(cc_nopeaks.flatten() ** 2)

    return (peak_height ** 2) / pce_energy * np.sign(peak_height), (peak_height ** 2), (
                pce_energy * np.sign(peak_height))

**YOUR CODE**

In [ ]:
import numpy as np

#read camera fingerprint
Fingerprint = np.genfromtxt('FINGERPRINT_D02.dat')
print('SIZE CAMERA FINGERPRINT: ', np.shape(Fingerprint))

#extract PRNU from image
imx = 'frame1.png'  #do not forget frame25.png frame50.png and frame75.png :-)
Noisex = Ft.NoiseExtractFromImage(imx, sigma=2.)
Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))
print('SIZE PRNU IMAGE: ', np.shape(Noisex))

if np.shape(Noisex) == np.shape(Fingerprint):
    print('Camera Fingerprint and Image PRNU size are the same. CASE: Perfectly aligned!')
    shift_range = [0, 0]
    C = Fu.crosscorr(Noisex1, Fingerprint)
else:
    print('Camera Fingerprint and Image PRNU size are NOT the same. CASE: NOT perfectly aligned!')
    Noisex1 = np.zeros_like(Fingerprint)
    Noisex1[:Noisex.shape[0], :Noisex.shape[1]] = Noisex
    shift_range = [Fingerprint.shape[0] - Noisex.shape[0], Fingerprint.shape[1] - Noisex.shape[1]]
    C = Fu.crosscorr(Noisex1, Fingerprint)

det, det0 = md.PCE(C, shift_range=shift_range)
print('PCE value before down-scaling inversion (i.e. upscaling)')
print("{0}: {1}".format('PCE', det['PCE']))

#convert Noisex and Fingerprint from numpy to tensor to use it with tfa.image.transform()
W_tensor = tf.expand_dims(tf.convert_to_tensor(Noisex, dtype=tf.float32), -1)
K_tensor = tf.expand_dims(tf.convert_to_tensor(Fingerprint, dtype=tf.float32), -1)

#to compute the PCE adapt this code
#size_frame_post = np.shape(W_tensor)
#C = circxcross2(Krs, W_tensor)
#ranges2 = [Fingerprint.shape[0] - size_frame_post[0] + 1, Fingerprint.shape[1] - size_frame_post[1] + 1]
#PCE_ks, _, _ = pce2((tf.squeeze(C)).numpy(), ranges2)
'''
YOUR CODE
[There are not real ranges of values for a0, a1, a2, etc.]
'''